# Lab 1

## Problem Statement: Accessing and interacting with the LLMs

In [ ]:
!pip install openai google-genai

In [ ]:
import time, os, getpass, csv
from typing import Dict, Any, Callable

Importing the secret keys

In [ ]:
# --- 1. load the file --------------------------------------------------------
keys = {}
with open("keys.csv") as f:
    for line in f:
        if line.strip():                     # skip blank lines
            k, v = line.strip().split(",", 1)
            keys[k] = v

# --- 2. export to environment ------------------------------------------------
for k in ("OPENAI_API_KEY", "GEMINI_API_KEY"):
    os.environ[k] = keys[k]

print("Keys imported and exported to environment.")

Keys imported and exported to environment.


Defining the query response functions for OpenAI and Gemini Models, followed by function to compare the responses

In [ ]:
# SDK imports
import openai
try:
    from google import genai
except Exception as e:
    genai = None

# Read API keys from environment variables (set in Step 2)
openai_api_key = os.getenv("OPENAI_API_KEY", "")
gemini_api_key = os.getenv("GEMINI_API_KEY", "")

prompt = "Explain the difference between Machine Learning and Generative AI in simple terms."

# --- Function Definitions for Real API Calls using SDKs ---

def query_openai(prompt, api_key):
    """Queries OpenAI using the official SDK."""
    try:
        # Modern OpenAI client style
        try:
            client = openai.OpenAI(api_key=api_key)
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=150,
                temperature=0.7
            )
            return response.choices[0].message.content.strip()
        except Exception:
            # Fallback legacy style
            openai.api_key = api_key
            resp = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=150,
                temperature=0.7
            )
            return resp.choices[0].message['content'].strip()
    except Exception as e:
        return f"OpenAI Error: {type(e).__name__} - {e}"

def query_gemini(prompt, api_key):
    """Queries Gemini using google-genai SDK, if available."""
    if genai is None:
        return "Gemini Error: google-genai SDK not installed or import failed."
    try:
        client = genai.Client(api_key=api_key)
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=prompt,
            config={"max_output_tokens": 150}
        )
        text = getattr(response, "text", None) or getattr(response, "outputs", None)
        if isinstance(text, str):
            return text.strip()
        if isinstance(text, list) and len(text) > 0:
            return str(text[0]).strip()
        return str(response).strip()
    except Exception as e:
        return f"Gemini Error: {type(e).__name__} - {e}"

# --- Comparison Function (MINIMAL OUTPUT) ---

def compare_responses():
    """Fetches real responses, measures latency, and prints only length and latency."""

    models: Dict[str, Callable] = {
        "OpenAI": lambda p: query_openai(p, openai_api_key),
        "Gemini": lambda p: query_gemini(p, gemini_api_key)
    }

    results: Dict[str, Any] = {}

    print("--- Fetching Responses (Using SDKs & Your API Keys) ---")
    for model_name, query_func in models.items():
        start_time = time.time()
        response_text = query_func(prompt).strip()
        latency = time.time() - start_time

        results[model_name] = {
            "latency": latency,
            "char_length": len(response_text)
        }
        print(f"Finished fetching response for {model_name}...")

    # Print the final Comparison Table
    print("\n" + "="*50)
    print(" LLM Length and Latency Comparison ")
    print("="*50)

    for model_name, data in results.items():
        print(f"\n--- {model_name} ---")
        print(f"Latency: {data['latency']:.4f}s")
        print(f"Character Length: {data['char_length']}")

    print("\n" + "="*50)

# Run comparison
if __name__ == "__main__":
    compare_responses()


--- Fetching Responses (Using SDKs & Your API Keys) ---
Finished fetching response for OpenAI...
Finished fetching response for Gemini...

 LLM Length and Latency Comparison 

--- OpenAI ---
Latency: 2.4734s
Character Length: 759

--- Gemini ---
Latency: 1.9105s
Character Length: 610

